In [1]:
#LOAD IN DATA FROM EXCEL
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Read data
data_path = "nurse_schedule_project2_data_large_VA.xlsx"
locations = pd.read_excel(data_path, sheet_name="locations", index_col = 0, header = None)
nurses = pd.read_excel(data_path, sheet_name="nurses", index_col = 0)
task_time = pd.read_excel(data_path, sheet_name="task_execution_time", index_col = 0).T

patients = pd.read_excel(data_path, sheet_name="patients", index_col = 0)
# Break comma separated strings into lists
for col in ['M', 'T', 'W','Th','F','S','Su']:
    patients[col] = patients[col].apply(lambda x: x.split(', ') if isinstance(x, str) else [])
for col in ['skillset']:
    nurses[col] = nurses[col].apply(lambda x: x.split(', ') if isinstance(x, str) else [])

# Transpose the dataframe to access patient information easily
patients = patients.T
nurses = nurses.T


# Sets and Parameters
N = nurses.columns.tolist()            # set of nurses
P = patients.columns.tolist()          # set of patients
L = list(locations.index)         # set of locations
D = ['M', 'T', 'W', 'Th', 'F', 'S', 'Su']  # set of days
T_tasks = task_time.columns.tolist() # set of tasks

# Parameters


# define a function to list all patients in a given location
def find_keys_with_inner_value(df, target_value):
    return list(df.T[df.T['location'] == target_value].index)


print(N, P, L, D, T_tasks)

['Nurse_1', 'Nurse_2', 'Nurse_3', 'Nurse_4', 'Nurse_5', 'Nurse_6', 'Nurse_7', 'Nurse_8', 'Nurse_9', 'Nurse_10', 'Nurse_11', 'Nurse_12', 'Nurse_13', 'Nurse_14', 'Nurse_15', 'Nurse_16', 'Nurse_17', 'Nurse_18', 'Nurse_19', 'Nurse_20', 'Nurse_21', 'Nurse_22', 'Nurse_23', 'Nurse_24', 'Nurse_25', 'Nurse_26', 'Nurse_27', 'Nurse_28', 'Nurse_29', 'Nurse_30', 'Nurse_31', 'Nurse_32', 'Nurse_33', 'Nurse_34', 'Nurse_35', 'Nurse_36', 'Nurse_37', 'Nurse_38', 'Nurse_39', 'Nurse_40', 'Nurse_41', 'Nurse_42', 'Nurse_43', 'Nurse_44', 'Nurse_45', 'Nurse_46', 'Nurse_47', 'Nurse_48', 'Nurse_49', 'Nurse_50'] ['Patient_1', 'Patient_2', 'Patient_3', 'Patient_4', 'Patient_5', 'Patient_6', 'Patient_7', 'Patient_8', 'Patient_9', 'Patient_10', 'Patient_11', 'Patient_12', 'Patient_13', 'Patient_14', 'Patient_15', 'Patient_16', 'Patient_17', 'Patient_18', 'Patient_19', 'Patient_20', 'Patient_21', 'Patient_22', 'Patient_23', 'Patient_24', 'Patient_25', 'Patient_26', 'Patient_27', 'Patient_28', 'Patient_29', 'Patient_3

In [ ]:
# City with needs on each day

# Total hours of needs on each day / city 

# 